In [0]:
!wget www.vision.caltech.edu/Image_Datasets/Caltech256/256_ObjectCategories.tar

In [0]:
!tar xf "256_ObjectCategories.tar"

In [0]:
!wget https://raw.githubusercontent.com/Ujjwal-9/Knowledge-Distillation/master/xception.py

In [0]:
!wget https://raw.githubusercontent.com/Ujjwal-9/Knowledge-Distillation/master/utils/image_preprocessing_ver1.py

--2019-02-26 03:46:32--  https://raw.githubusercontent.com/Ujjwal-9/Knowledge-Distillation/master/utils/image_preprocessing_ver1.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44072 (43K) [text/plain]
Saving to: ‘image_preprocessing_ver1.py’

image_preprocessing 100%[===================>]  43.04K  --.-KB/s    in 0.01s   

2019-02-26 03:46:32 (4.19 MB/s) - ‘image_preprocessing_ver1.py’ saved [44072/44072]



In [0]:
import numpy as np
from tqdm import tqdm


# use non standard flow_from_directory
from image_preprocessing_ver1 import ImageDataGenerator
# it outputs not only x_batch and y_batch but also image names

from keras.models import Model ,load_model
from xception import Xception, preprocess_input

In [0]:
data_dir = 'img/'

In [0]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', 
    target_size=(299, 299),
    batch_size=50, shuffle=False
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(299, 299),
    batch_size=50, shuffle=False
)

Found 5000 images belonging to 10 classes.
Found 3000 images belonging to 10 classes.


In [0]:
model = Xception(classes=10)
model.load_weights('/content/gdrive/My Drive/Colab Notebooks/classification/xception_kd/xception.h5')
# remove softmax
model.layers.pop()
model = Model(model.input, model.layers[-1].output)
# now model outputs logits


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
batches = 0
train_logits = {}

for x_batch, y_batch, name_batch in tqdm(train_generator):
    
    batch_logits = model.predict_on_batch(x_batch)
    
    for i, n in enumerate(name_batch):
        #print(n)
        train_logits[n] = batch_logits[i]
    
    batches += 1
    if batches >= 5000//50: # 5000/64
        break



In [0]:
batches = 0
val_logits = {}
numb = 0
for x_batch, _, name_batch in tqdm(val_generator):
    
    batch_logits = model.predict_on_batch(x_batch)

    for i, n in enumerate(name_batch):
        val_logits[n] = batch_logits[i]
    
    batches += 1
    if batches >= 3000//50: # 3000/64
        break

In [0]:
np.save('train_logits.npy', train_logits)
np.save('val_logits.npy', val_logits)

In [0]:
!cp "train_logits.npy" "/content/gdrive/My Drive/Colab Notebooks/classification/xception_kdtrain_logits.npy"
!cp "val_logits.npy" "/content/gdrive/My Drive/Colab Notebooks/classification/xception_kd/val_logits.npy"

Logits with softmax

In [0]:
# Define a manual softmax function
def softmax(x):
    return np.exp(x)/(np.exp(x).sum())

In [0]:
import cv2
from image_preprocessing_ver1 import ImageDataGenerator ,img_to_array,load_img

image = cv2.imread("/content/img/train/bird/1350.png")
image = cv2.resize(image, (299,299) )
#image /= 255.0
#image -= 0.5
#image *= 2.0
#image = img_to_array(image)
#model.predict(image)
#image

input_shape = (299, 299, 3)
#img = load_img("/content/img/train/bird/1350.png")
x = img_to_array(image, data_format='channels_last')
x /= 255.0
x -= 0.5
x *= 2.0
#x = ImageDataGenerator.standardize(x)
x = np.reshape(x, (-1, 299, 299 ,3) )
logito = model.predict(x)
logito



array([[-0.3852763 ,  5.040748  , -2.071821  , -0.5846154 ,  2.1038177 ,
        -0.3661116 , -1.685475  ,  0.07440768, -0.88533896, -1.8386288 ]],
      dtype=float32)

In [0]:
np.argmax(logito)

1

In [0]:
batches = 0
train_logits = {}

for x_batch, y_batch, name_batch in tqdm(train_generator):
    
    batch_logits = model.predict_on_batch(x_batch)
    
    for i, n in enumerate(name_batch):
      #  print("label")
      #  print(x_batch[i])
      #  print(y_batch[i])
      #  print(n)
      #  print("normal logit")
      #  print(batch_logits[i])
      #  print(np.sum(batch_logits[i]))
      #  print("softmax logit")
        print(softmax(batch_logits[i]))
        print(np.sum(softmax(batch_logits[i])))
      #  print("softmax logit temperature")
      #  print(softmax(batch_logits[i])/3)
      #  print(np.sum(softmax(batch_logits[i]/3)))
      #  print("====================================")
        train_logits[n] = softmax(batch_logits[i]) #/3batch_logits[i]
    
    batches += 1
    if batches >= 5000//50: # 5000/64
        break








0it [00:00, ?it/s]





1it [00:01,  1.15s/it]

[5.8515687e-03 9.7238189e-01 1.3070925e-03 9.2245772e-04 9.0958802e-03
 1.5379322e-03 6.0144002e-03 9.5727155e-04 1.2075916e-03 7.2383886e-04]
0.99999994
[1.9749471e-04 9.9741775e-01 4.5733675e-04 9.2214970e-05 1.9618636e-04
 1.4277651e-04 1.9945855e-04 1.1268437e-03 9.0136069e-05 7.9743309e-05]
0.99999994
[3.6532295e-04 9.9670160e-01 1.4183173e-04 2.9246547e-04 7.2281895e-04
 6.5197586e-04 1.4669127e-04 6.2206702e-04 1.4404827e-04 2.1125107e-04]
1.0000001
[1.5480460e-03 9.8876083e-01 7.7577768e-04 2.6452093e-04 2.0110416e-03
 1.4216752e-03 2.5718096e-03 1.5208585e-03 6.4654823e-04 4.7881907e-04]
0.9999999
[1.1489063e-04 9.9798214e-01 9.8800039e-05 1.6231957e-04 4.0483093e-04
 1.2945753e-04 1.4780695e-04 7.6587382e-04 6.9888738e-05 1.2406449e-04]
1.0000001
[1.3920210e-03 9.9783957e-01 3.0459774e-05 3.8136790e-05 6.9737158e-05
 2.6452970e-05 3.2458564e-05 5.1441946e-04 3.0654272e-05 2.6011237e-05]
0.9999999
[1.5102517e-03 9.9378449e-01 8.8408432e-04 2.5178958e-04 5.6394818e-04
 7.636701







2it [00:02,  1.16s/it]

[6.70430250e-03 9.89301085e-01 2.28132398e-04 1.06053354e-04
 2.98179773e-04 1.69557374e-04 7.23375764e-04 1.96943921e-03
 2.71802157e-04 2.28015648e-04]
0.99999994
[1.8000694e-02 3.7944740e-01 8.0696127e-04 4.3688398e-03 9.2263082e-03
 1.1641397e-03 1.8620970e-02 5.6689090e-01 9.3501841e-04 5.3877640e-04]
1.0
[5.8368747e-03 9.7767299e-01 4.9199513e-04 5.4875255e-04 2.3288977e-04
 2.7021300e-03 4.9261623e-03 1.9321684e-03 5.2851187e-03 3.7091013e-04]
1.0
[6.6199391e-05 9.9961764e-01 2.4280960e-05 2.1406448e-05 1.3228539e-04
 1.8204939e-05 2.7013342e-05 3.6822672e-05 3.5756591e-05 2.0331014e-05]
1.0
[9.9444063e-05 9.9501377e-01 1.5339111e-04 4.4451078e-04 1.0225471e-03
 2.8749389e-04 8.2194543e-04 1.6617724e-03 2.5924557e-04 2.3586895e-04]
0.99999994
[1.0750700e-03 9.6256250e-01 1.1888648e-03 1.1303598e-03 4.9531665e-03
 4.8881345e-03 4.5649898e-03 1.7927239e-02 4.6245116e-04 1.2472795e-03]
1.0000001
[1.4065651e-03 4.8179218e-01 1.0649078e-03 1.0551200e-03 3.0826595e-02
 5.7380395e-03 5







3it [00:03,  1.16s/it]

...

[9.9999344e-01 5.2461826e-07 1.2651221e-07 4.9658084e-08 5.1739318e-07
 3.1527466e-07 5.5623264e-07 9.1365035e-07 2.2858235e-06 1.2650704e-06]
1.0
[9.9967253e-01 1.1029291e-05 2.5773137e-05 3.4010429e-06 2.0308431e-05
 8.6942255e-06 1.7345243e-05 1.5545571e-05 1.1771391e-04 1.0759979e-04]
0.9999999
[9.9998528e-01 3.9618618e-07 7.6335562e-07 1.5379939e-07 1.3120780e-06
 5.1438985e-07 6.0070533e-07 6.2956656e-07 6.0440634e-06 4.3568775e-06]
1.0
[9.9995977e-01 1.1928556e-06 2.6613736e-06 3.3979103e-07 1.3897513e-06
 1.1599020e-06 2.4683900e-06 1.6571867e-06 2.4282890e-05 5.0593658e-06]
0.99999994
[9.9999201e-01 8.1094072e-07 4.2488659e-07 1.7300454e-07 6.7483757e-07
 3.0898011e-07 8.0566588e-07 4.4045575e-07 2.1813833e-06 2.1457931e-06]
1.0
[9.9998933e-01 4.1579040e-07 4.5370319e-07 9.4410382e-08 4.7363412e-07
 7.3758184e-07 9.1625623e-07 7.5903773e-07 2.5057966e-06 4.3086234e-06]
0.99999994
[9.9982631e-01 6.0908684e-07 9.5525047e-07 9.4646786e-08 2.6760126e-06
 7.2159901e-06 1.6576170e-0







98it [01:52,  1.14s/it]

[9.9990481e-01 5.4923134e-06 1.5286574e-05 4.3100299e-06 2.1834474e-06
 2.8987238e-06 3.1095728e-06 8.0939853e-06 4.6174620e-05 7.6407796e-06]
1.0
[9.9911141e-01 1.4607797e-06 4.9772251e-07 6.1343044e-08 5.0622026e-07
 6.1497553e-07 6.7057596e-07 1.9872227e-06 8.8130700e-04 1.5471111e-06]
1.0
[9.9990040e-01 2.5300908e-06 8.3488094e-06 7.6802928e-07 1.3244082e-05
 3.9698543e-06 1.0817519e-05 3.0146173e-06 2.0872982e-05 3.6023084e-05]
0.99999994
[9.99927342e-01 3.48340336e-07 6.46407784e-07 1.08923416e-07
 8.44356748e-07 5.78654522e-07 1.62181789e-06 2.69410322e-07
 9.97182360e-06 5.82641987e-05]
1.0
[9.9989653e-01 2.1756653e-06 6.6713988e-06 3.8355805e-07 4.2373063e-06
 2.9681526e-06 8.0453292e-06 1.7272364e-06 4.1938045e-05 3.5314551e-05]
1.0
[9.9997407e-01 1.5027766e-06 1.7575184e-06 2.7072500e-07 9.6983786e-07
 1.1343323e-06 2.0489733e-06 1.0928894e-06 5.4546517e-06 1.1592728e-05]
0.9999999
[9.9993449e-01 2.5459574e-06 1.9001310e-06 4.2434161e-07 1.7414883e-06
 1.1379843e-06 1.181148







99it [01:53,  1.14s/it]

[2.1436354e-03 9.4255793e-01 4.7295971e-04 2.3161625e-03 4.6406507e-02
 1.4019166e-03 4.4324785e-04 2.5709260e-03 1.1592807e-03 5.2748370e-04]
1.0000001
[2.2840455e-04 9.9857277e-01 2.7103935e-04 1.2891160e-04 1.7968024e-04
 1.7544960e-04 1.0548004e-04 8.0394646e-05 1.9916588e-04 5.8743764e-05]
1.0
[1.6509452e-04 9.9847698e-01 1.0055159e-04 4.8801501e-04 1.8156567e-04
 1.2025402e-04 5.5135064e-05 2.0301579e-04 1.7128773e-04 3.8155871e-05]
1.0000001
[7.0272712e-05 9.9967402e-01 3.9396858e-05 1.6124695e-05 3.8985247e-05
 4.0609728e-05 3.7248552e-05 4.9933304e-05 2.1236294e-05 1.2108549e-05]
0.9999999
[4.38167073e-04 9.98396397e-01 1.64277822e-04 8.07666293e-05
 1.51407658e-04 2.50261277e-04 1.09531473e-04 1.70465049e-04
 1.18921576e-04 1.19775279e-04]
0.9999999
[5.0662663e-03 9.8010606e-01 1.7541504e-03 1.0445399e-03 2.5077001e-03
 2.3753990e-03 2.6027518e-03 1.7390801e-03 2.0807853e-03 7.2327454e-04]
1.0
[1.6323308e-03 9.9021846e-01 9.6728501e-04 3.8094568e-04 8.4041158e-04
 1.6391315e-

In [0]:
batches = 0
val_logits = {}
numb = 0
for x_batch, _, name_batch in tqdm(val_generator):
    
    batch_logits = model.predict_on_batch(x_batch)

    for i, n in enumerate(name_batch):
        val_logits[n] = softmax(batch_logits[i]) #/3batch_logits[i]
    
    batches += 1
    if batches >= 3000//50: # 3000/64
        break








0it [00:00, ?it/s]






1it [00:01,  1.10s/it]






2it [00:02,  1.10s/it]






3it [00:03,  1.10s/it]






4it [00:04,  1.10s/it]






5it [00:05,  1.10s/it]






6it [00:06,  1.11s/it]






7it [00:07,  1.11s/it]






8it [00:08,  1.11s/it]






9it [00:09,  1.11s/it]






10it [00:11,  1.11s/it]






11it [00:12,  1.11s/it]






12it [00:13,  1.11s/it]






13it [00:14,  1.11s/it]






14it [00:15,  1.11s/it]






15it [00:16,  1.11s/it]






16it [00:17,  1.11s/it]






17it [00:18,  1.11s/it]






18it [00:19,  1.11s/it]






19it [00:21,  1.11s/it]






20it [00:22,  1.11s/it]






21it [00:23,  1.11s/it]






22it [00:24,  1.12s/it]






23it [00:25,  1.12s/it]






24it [00:26,  1.12s/it]






25it [00:27,  1.12s/it]






26it [00:28,  1.11s/it]






27it [00:29,  1.11s/it]






28it [00:31,  1.12s/it]






29it [00:32,  1.11s/it]






30it [00:33,  1.11s/it]






31it [00:34,  1.11s/it]






32it [00:35,  1.11s/it]






33it [00:36,  1.1

In [0]:
np.save('train_logits_temp1.npy', train_logits)
np.save('val_logits_temp1.npy', val_logits)

In [0]:
!cp "train_logits_temp1.npy" "/content/gdrive/My Drive/Colab Notebooks/classification/xception_kdtrain_logits_temp1.npy"
!cp "val_logits_temp1.npy" "/content/gdrive/My Drive/Colab Notebooks/classification/xception_kd/val_logits_temp1.npy"